In [ ]:
# Install required packages
pip install psutil

In [ ]:
# Install required packages
!pip install --quiet gspread pandas

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import gspread
import pandas as pd
import numpy as np
import torch
import re
from google.colab import files
import tracemalloc
import time
import psutil, os
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import nltk
from scipy.special import softmax

In [ ]:
uploaded = files.upload()

In [ ]:
fd = pd.read_excel("finaldata.xlsx")
fd

In [ ]:
Bangladesh_list = fd[fd['Country'] == 'Bangladesh']
Bangladesh_list = Bangladesh_list.Location.unique().tolist()
print(len(Bangladesh_list))
print(Bangladesh_list)

In [ ]:
India_list = fd[fd['Country'] == 'India']
India_list = India_list.Location.unique().tolist()
print(len(India_list))
print(India_list)

In [ ]:
Canada_list = fd[fd['Country'] == 'Canada']
Canada_list = Canada_list.Location.unique().tolist()
print(len(Canada_list))
print(Canada_list)

In [ ]:
Us_list = fd[fd['Country'] == 'United States']
Us_list = Us_list.Location.unique().tolist()
print(len(Us_list))
print(Us_list)

In [ ]:
Uk_list = fd[fd['Country'] == 'United Kingdom']
Uk_list = Uk_list.Location.unique().tolist()
print(len(Uk_list))
print(Uk_list)

In [ ]:
France_list = fd[fd['Country'] == 'France']
France_list = France_list.Location.unique().tolist()
print(len(France_list))
print(France_list)

In [ ]:
Australia_list = fd[fd['Country'] == 'Australia']
Australia_list = Australia_list.Location.unique().tolist()
print(len(Australia_list))
print(Australia_list)

In [ ]:
Pakistan_list = fd[fd['Country'] == 'Pakistan']
Pakistan_list = Pakistan_list.Location.unique().tolist()
print(len(Pakistan_list))
print(Pakistan_list)

In [ ]:
Netherlands_list = fd[fd['Country'] == 'Netherlands']
Netherlands_list = Netherlands_list.Location.unique().tolist()
print(len(Netherlands_list))
print(Netherlands_list)

In [ ]:
Germany_list = fd[fd['Country'] == 'Germany']
Germany_list = Germany_list.Location.unique().tolist()
print(len(Germany_list))
print(Germany_list)

In [ ]:
uploaded = files.upload()

In [ ]:
coviddata = pd.read_excel("Country_dataset.xlsx")
coviddata

In [ ]:
coviddata['increase_confirmed'] = coviddata['Confirmed 22nd June']-coviddata['Confirmed 1st april']
coviddata['increase_confirmed'] = coviddata['increase_confirmed']/coviddata['Population']
coviddata['increase_confirmed']

In [ ]:
coviddata['increase_deaths'] = coviddata['Deaths 22nd June']-coviddata['Deaths 1st april']
coviddata['increase_deaths'] = coviddata['increase_deaths']/coviddata['Population']
coviddata['increase_deaths']

In [ ]:
max3 = coviddata['increase_confirmed'].max()
min3 = coviddata['increase_confirmed'].min()
max4 = coviddata['increase_deaths'].max()
min4 = coviddata['increase_deaths'].min()

In [ ]:
print(max3,min3,max4,min4)

In [ ]:
coviddata['normalized_increase_confirmed'] = coviddata['increase_confirmed'].apply(lambda x: (((x-min3)+0.0001)/((max3-min3)+0.0001)))

In [ ]:
coviddata['normalized_increase_deaths'] = coviddata['increase_deaths'].apply(lambda y: (((y-min4)+0.0001)/((max4-min4)+0.0001)))

In [ ]:
coviddata['intensity'] = (coviddata['normalized_increase_confirmed']+coviddata['normalized_increase_deaths'])/2
# 'intensity' corresponds to the Severity Value (SV) as described in the manuscript.
coviddata[['Country','intensity']]

## Important NOTE

The cells below require a `Location` column derived from hydrated Twitter data.

Due to Twitter/X Developer Policy and privacy constraints, raw tweet text and
user location are not included in the public release.

To reproduce the results:
1. Hydrate the published Tweet IDs using the Twitter API to retrieve the  
   **tweet_text** and **Location** fields.
2. Run *Final_code_for_publication_1.ipynb* to compute local sentiment labels
   and sentiment scores.
3. Store the resulting dataset in a Google Sheet using your own Google
   Service Account credentials.

**All subsequent analyses assume these steps have been completed.**

In our private execution, we store the following fields in a Google Sheet:
- `tweet_text`
- `Tweet_ID`
- `Location`
- `pred_label` (local sentiment)
- `final_tweet_logit` (Sentiment Score)

However, **credentials and spreadsheet keys are NOT included** in this public
repository for ethical reasons and to comply with Twitter/X Terms of Service.


In [ ]:
# Use the path to your credentials file in Drive
gc = gspread.service_account(filename="PATH_TO_YOUR_CREDENTIALS.json")

In [ ]:
# Open your Google Sheet by key
sh = gc.open_by_key("YOUR_SPREADSHEET_KEY")

In [ ]:
worksheet1 = sh.get_worksheet(2)

In [ ]:
# Get records
res1 = worksheet1.get_all_records()

In [ ]:
data1=pd.DataFrame(res1)
data1

In [ ]:
positive = data1[data1['pred_label']=='positive']
neutral = data1[data1['pred_label']=='neutral']
negative = data1[data1['pred_label']=='negative']

In [ ]:
print("positive:",len(positive))
print("neutral:",len(neutral))
print("negative :",len(negative))

In [ ]:
data1['Location']=data1['Location'].str.lower()
#list1= data1.Location.unique().tolist()
#print(list1)

In [ ]:
data1['Country']=''
data1.loc[data1['Location'].isin(India_list),'Country']='india'
data1.loc[data1['Location'].isin(Canada_list),'Country']='canada'
data1.loc[data1['Location'].isin(Us_list),'Country']='us'
data1.loc[data1['Location'].isin(Uk_list),'Country']='united kingdom'
data1.loc[data1['Location'].isin(France_list),'Country']='france'
data1.loc[data1['Location'].isin(Bangladesh_list),'Country']='bangladesh'
data1.loc[data1['Location'].isin(Australia_list),'Country']='australia'
data1.loc[data1['Location'].isin(Pakistan_list),'Country']='pakistan'
data1.loc[data1['Location'].isin(Netherlands_list),'Country']='netherlands'
data1.loc[data1['Location'].isin(Germany_list),'Country']='germany'
data1[['Location','Country']]

In [ ]:
finaldata = data1

In [ ]:
finaldata

In [ ]:
finaldata.replace('', np.nan, inplace=True)
finaldata = finaldata.dropna(subset=['Country'])

In [ ]:
finaldata

In [ ]:
finaldata['SV']=''
finaldata.loc[finaldata['Country']=='canada','SV']=0.513569
finaldata.loc[finaldata['Country']=='india','SV']=0.683371
finaldata.loc[finaldata['Country']=='us','SV']=0.578305
finaldata.loc[finaldata['Country']=='united kingdom','SV']=0.278029
finaldata.loc[finaldata['Country']=='bangladesh','SV']=0.228202
finaldata.loc[finaldata['Country']=='france','SV']=0.855757
finaldata.loc[finaldata['Country']=='australia','SV']=0.154538
finaldata.loc[finaldata['Country']=='pakistan','SV']=0.231838
finaldata.loc[finaldata['Country']=='netherlands','SV']=0.756880
finaldata.loc[finaldata['Country']=='germany','SV']=0.637414

In [ ]:
finaldata

In [ ]:
ordered_cats = ["negative", "neutral", "positive"]
finaldata["sentiment_ord"] = pd.Categorical(finaldata["pred_label"],
                                    categories=ordered_cats,
                                    ordered=True)

In [ ]:
# Check counts by class
print(finaldata["sentiment_ord"].value_counts(dropna=False))

In [ ]:
positive = finaldata[finaldata['pred_label']=='positive']
neutral = finaldata[finaldata['pred_label']=='neutral']
negative = finaldata[finaldata['pred_label']=='negative']

In [ ]:
print("positive:",len(positive))
print("neutral:",len(neutral))
print("negative :",len(negative))

In [ ]:
finaldata

In [ ]:
# Convert SV to numeric
finaldata["SV"] = pd.to_numeric(finaldata["SV"], errors="coerce")

In [ ]:
print(finaldata["SV"].dtype)
print(finaldata["SV"].unique()[:10])

In [ ]:
!pip install bambi pymc arviz

In [ ]:
import bambi as bmb
import pymc as pm
import arviz as az
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import spearmanr
from sklearn.metrics import accuracy_score

In [ ]:
np.random.seed(123)
random.seed(123)

In [ ]:
# OPTION 1: Run this cell if you want an overall train–test split across all countries
train_data, test_data = train_test_split(
    finaldata,
    test_size=0.30,
    stratify=finaldata["sentiment_ord"],
    random_state=123
)


In [ ]:
# OPTION 2: Run this cell if you want a country-level train–test split
holdout_countries = ["india", "netherlands"]   # example
train_data = finaldata[~finaldata["Country"].isin(holdout_countries)].copy()
test_data  = finaldata[ finaldata["Country"].isin(holdout_countries)].copy()

In [ ]:
print(len(train_data), len(test_data))

In [ ]:
process = psutil.Process(os.getpid())

In [ ]:
model = bmb.Model(
    "sentiment_ord ~ final_tweet_logit * SV + (1 | Country)",
    train_data,
    family="cumulative"
)

In [ ]:
tracemalloc.start()
start_train = time.time()

idata = model.fit(
    draws=1000,
    tune=1500,
    chains=2,
    cores=1,
    target_accept=0.97,
    random_seed=123
)

train_time = time.time() - start_train

current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"Training time: {train_time:.2f} seconds")
print(f"Peak memory during training: {peak / (1024 * 1024):.2f} MB")


In [ ]:
tracemalloc.start()
start_test = time.time()

# Set sample_new_groups=True for OPTION 2 (country-level train–test split);
# keep it False for OPTION 1 (overall train–test split).

model.predict(idata, data=test_data, kind="response", sample_new_groups=True)

inference_time = time.time() - start_test
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"Inference time: {inference_time:.4f} seconds")
print(f"Peak memory during prediction: {peak / (1024 * 1024):.2f} MB")


In [ ]:
print(idata.posterior_predictive)

In [ ]:
# Extract samples: shape (chains, draws, n_obs)
pp_samples = idata.posterior_predictive["sentiment_ord"].values
C, D, N = pp_samples.shape
K = 3  # classes 0..2

In [ ]:
idata.posterior_predictive["sentiment_ord"].shape

In [ ]:
# Reshape to (S, N) where S = C*D (all posterior predictive draws)
samples = pp_samples.reshape(-1, N)   # shape (S, N)

In [ ]:
# Compute class probabilities by frequency across draws
probs = np.zeros((N, K))
for k in range(K):
    probs[:, k] = (samples == k).mean(axis=0)  # P(y=k) per observation

In [ ]:
pred_argmax = probs.argmax(axis=1)

In [ ]:
# Expected ordinal value
expected_vals = (probs * np.arange(K)).sum(axis=1)  # shape (N,)

In [ ]:
# True labels
y_true = test_data["sentiment_ord"].cat.codes.values  # 0/1/2

In [ ]:
# Metrics
# MAE, MSE, Spearman
mae_expected = mean_absolute_error(y_true, expected_vals)
mse_expected = mean_squared_error(y_true, expected_vals)
spearman_expected, spearman_p = spearmanr(y_true, expected_vals)

In [ ]:
print("=== Expected-value ===")
print("MAE:", mae_expected)
print("MSE:", mse_expected)
print("Spearman:", spearman_expected, spearman_p)

In [ ]:
#print("MAE_argmax:", mean_absolute_error(y_true, pred_argmax))
print("Accuracy:",accuracy_score(y_true, pred_argmax))

In [ ]:
brier = np.mean(np.sum((probs - np.eye(K)[y_true])**2, axis=1))
print("Brier score:", brier)

In [ ]:
def compute_ece(y_true, probs, n_bins=10):
    confidences = probs.max(axis=1)
    preds = probs.argmax(axis=1)
    ece = 0.0
    bins = np.linspace(0.0, 1.0, n_bins+1)
    for i in range(n_bins):
        mask = (confidences > bins[i]) & (confidences <= bins[i+1])
        if mask.sum() == 0:
            continue
        acc = (preds[mask] == y_true[mask]).mean()
        conf = confidences[mask].mean()
        ece += (mask.sum() / len(y_true)) * abs(acc - conf)
    return ece

ece = compute_ece(y_true, probs, n_bins=10)
print("ECE:", ece)

In [ ]:
def bootstrap_ci_mae(y_true, expected_vals, B=1000):
    n = len(y_true)
    stats = []
    for _ in range(B):
        idx = np.random.choice(n, n, replace=True)
        stats.append(mean_absolute_error(y_true[idx], expected_vals[idx]))
    return np.percentile(stats, [2.5, 97.5])

print("MAE_expected 95% CI:", bootstrap_ci_mae(y_true, expected_vals, B=1000))

In [ ]:
model_full = bmb.Model(
    "sentiment_ord ~ final_tweet_logit * SV + (1 | Country)",
    finaldata,
    family="cumulative"
)



In [ ]:

tracemalloc.start()
start_trainf = time.time()

idata_full = model_full.fit(
    draws=1000,
    tune=1500,
    chains=2,
    cores=1,
    target_accept=0.97,
    random_seed=123
)

train_timef = time.time() - start_trainf
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"Full dataset Training time: {train_timef:.2f} seconds")
print(f"Peak memory during full dataset training: {peak / (1024 * 1024):.2f} MB")



In [ ]:
model_full.predict(
    idata_full,
    data=finaldata,
    kind="response"
)


In [ ]:
idata_full.posterior_predictive["sentiment_ord"].shape

In [ ]:
pp_samples_full = idata_full.posterior_predictive["sentiment_ord"].values
C, D, N = pp_samples_full.shape   # N = size of full dataset
K = 3

# reshape
samples_full = pp_samples_full.reshape(-1, N)

# compute probabilities
probs_full = np.zeros((N, K))
for k in range(K):
    probs_full[:, k] = (samples_full == k).mean(axis=0)

# expected ordinal intensity
#expected_vals_full = (probs_full * np.arange(K)).sum(axis=1)


In [ ]:
df = finaldata.copy()
df['p_neg'] = probs_full[:, 0]
df['p_neu'] = probs_full[:, 1]
df['p_pos'] = probs_full[:, 2]
df['max_p'] = probs_full.max(axis=1)
df['pred'] = probs_full.argmax(axis=1)  # hard model label 0/1/2
df['ordinal_label'] = pd.Categorical.from_codes(df['pred'], categories=['negative','neutral','positive'])

In [ ]:
def make_intensity_labels(df):
    out = df.copy()

    class_names = ['NEGATIVE', 'NEUTRAL', 'POSITIVE']
    prob_cols   = ['p_neg', 'p_neu', 'p_pos']

    # Two new columns: one for median thresholding and one for mean thresholding
    out['intensity_label_median'] = None
    out['intensity_label_mean']   = None

    for k, (cls, colname) in enumerate(zip(class_names, prob_cols)):

        mask_class = (out['pred'] == k)

        # -------- Median threshold --------
        thr_median = out.loc[mask_class, colname].median()
        print(thr_median)

        out.loc[mask_class & (out[colname] >= thr_median),
                'intensity_label_median'] = f"{cls} WITH HIGH INTENSITY"
        out.loc[mask_class & (out[colname] < thr_median),
                'intensity_label_median'] = f"{cls} WITH LOW INTENSITY"

        # -------- Mean threshold --------
        thr_mean = out.loc[mask_class, colname].mean()
        print(thr_mean)

        out.loc[mask_class & (out[colname] >= thr_mean),
                'intensity_label_mean'] = f"{cls} WITH HIGH INTENSITY"
        out.loc[mask_class & (out[colname] < thr_mean),
                'intensity_label_mean'] = f"{cls} WITH LOW INTENSITY"

    return out


In [ ]:
# Apply the function
df_median = make_intensity_labels(df)

In [ ]:
df_median['intensity_label_mean'].value_counts()

In [ ]:
df_median['intensity_label_median'].value_counts()

In [ ]:
df_median

In [ ]:
df_median['Tweet_ID'] = df_median['Tweet_ID'].astype(str)

In [ ]:
df_median.to_excel("ordinalfinal.xlsx", index=False)

In [ ]:
files.download("ordinalfinal.xlsx")

In [ ]:
summary = az.summary(idata_full, var_names=["SV", "final_tweet_logit", "final_tweet_logit:SV"], round_to=2)
print(summary)

In [ ]:
thresholds_summary = az.summary(
    idata_full,
    var_names=["threshold"],
    round_to=2,
)
print(thresholds_summary)

In [ ]:
group_summary = az.summary(
    idata_full,
    var_names=["1|Country"],
    round_to=2,
)
print(group_summary)


In [ ]:
group_summary1 = az.summary(
    idata_full,
    var_names=["1|Country_sigma"],
    round_to=2,
)
print(group_summary1)